# Metadata

**L1 Taxonomy** - Software Architecture & Design

**L2 Taxonomy** - Event-Driven Architecture

**Subtopic** - Simulating event sourcing: storing events and rebuilding state by replaying them (e.g. list of account transactions to reconstruct balance)

**Use Case** - Implement an event sourcing system in Python that simulates a bank account. The system should be able to store a list of account transactions (deposits and withdrawals) as events and reconstruct the account balance by replaying these events. The system should also be able to handle new transactions, add them to the event list, and update the account balance accordingly.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
```


# Prompt

You are tasked with implementing an event driven system for a simplified bank account that processes transactions and maintains account state through event replay. The system must store all transactions as immutable events and reconstruct the current balance by replaying these events in chronological order. The account starts with zero balance and processes deposits and withdrawals while maintaining a complete audit trail.

## Input Format

The function receives a list of transaction commands where each command is a tuple containing:
- Transaction type: "deposit" or "withdraw"
- Amount: positive integer representing the transaction amount
- Transaction ID: unique integer identifier for the transaction

Example: [("deposit", 1000, 1), ("withdraw", 300, 2), ("deposit", 50, 3)]

## Output Format

Return a dictionary containing:
- "final_balance": integer representing the current account balance
- "transaction_count": total number of processed transactions
- "event_log": list of dictionaries representing stored events in chronological order
- "balance_history": list of balance values after each transaction

Each event in event_log contains: {"transaction_id", "type", "amount", "resulting_balance"}

## Examples

Example 1:

Input:

```
[("deposit", 1000, 1), ("withdraw", 300, 2)]
```

Output:

```
{
  "final_balance": 700,
  "transaction_count": 2,
  "event_log": [
    {"transaction_id": 1, "type": "deposit", "amount": 1000, "resulting_balance": 1000},
    {"transaction_id": 2, "type": "withdraw", "amount": 300, "resulting_balance": 700}
  ],
  "balance_history": [1000, 700]
}
```

Example 2:

Input:

```
[("deposit", 500, 1), ("deposit", 200, 2), ("withdraw", 100, 3)]
```

Output:

```
{
  "final_balance": 600,
  "transaction_count": 3,
  "event_log": [
    {"transaction_id": 1, "type": "deposit", "amount": 500, "resulting_balance": 500},
    {"transaction_id": 2, "type": "deposit", "amount": 200, "resulting_balance": 700},
    {"transaction_id": 3, "type": "withdraw", "amount": 100, "resulting_balance": 600}
  ],
  "balance_history": [500, 700, 600]
}
```

# Requirements

## Explicit Requirements

1. Process transactions in the exact order provided in the input list
2. Store each transaction as an immutable event with transaction_id, type, amount, and resulting_balance
3. Calculate final balance by replaying all events from the beginning
4. Maintain balance_history showing account balance after each transaction
5. Handle both deposit and withdraw transaction types
6. Return all required fields in the output dictionary
7. It should be an event-driven system

## Implicit Requirements

1. Account balance must never be stored directly - always derived from event replay
2. Events must be stored in chronological order matching input sequence
3. Each transaction must generate exactly one event
4. Balance calculation must start from zero and process events sequentially
5. Event log must preserve complete transaction history
6. System must demonstrate true event sourcing principles

## Function Signature

```python
def process_bank_transactions(transactions):
    # transactions: list of tuples (type, amount, transaction_id)
    # returns: dictionary with final_balance, transaction_count, event_log, balance_history
    pass
```

## Edge Cases

1. If the transaction list is empty, process it by returning zero balance, empty event log, and empty balance history.
2. For a single transaction, process it normally and return the appropriate values.  
3. When multiple transactions share the same type, process each one independently.  
4. If a withdrawal results in a negative balance, allow it and continue processing.  
5. Treat zero-amount transactions as valid, even though they have no impact.  
6. Process all transactions even if their IDs are duplicated.

## Constraints

- Transaction amounts are non-negative integers (0 <= amount <= 1000000)
- Transaction IDs are positive integers (1 <= transaction_id <= 1000000)
- Maximum 1000 transactions per input
- Transaction types are limited to "deposit" and "withdraw"
- All monetary values are integers (no decimal handling required)
- Input list contains valid tuples with exactly 3 elements each
- Your response and code must be complete, else it isn't acceptable.

In [ ]:
# code

from typing import List, Dict, Tuple, Any
from dataclasses import dataclass
from abc import ABC, abstractmethod


@dataclass
class BankEvent:
    """Immutable event representing a bank transaction."""

    transaction_id: int
    type: str
    amount: int
    resulting_balance: int


class EventStore:
    """Stores and manages immutable events."""

    def __init__(self):
        self._events: List[BankEvent] = []

    def append_event(self, event: BankEvent) -> None:
        """Add a new event to the store."""
        self._events.append(event)

    def get_all_events(self) -> List[BankEvent]:
        """Retrieve all events in chronological order."""
        return self._events.copy()

    def get_event_count(self) -> int:
        """Get total number of events."""
        return len(self._events)


class Command(ABC):
    """Abstract base class for commands."""

    @abstractmethod
    def execute(self, current_balance: int) -> Tuple[int, str]:
        """Execute the command and return new balance and event type."""
        pass


class DepositCommand(Command):
    """Command for deposit transactions."""

    def __init__(self, amount: int):
        self.amount = amount

    def execute(self, current_balance: int) -> Tuple[int, str]:
        return current_balance + self.amount, "deposit"


class WithdrawCommand(Command):
    """Command for withdrawal transactions."""

    def __init__(self, amount: int):
        self.amount = amount

    def execute(self, current_balance: int) -> Tuple[int, str]:
        return current_balance - self.amount, "withdraw"


class BankAccount:
    """Event-driven bank account that maintains state through event replay."""

    def __init__(self):
        self.event_store = EventStore()

    def process_transaction(self, transaction_type: str, amount: int,
                            transaction_id: int) -> None:
        """Process a single transaction and store the resulting event."""
        # Create appropriate command
        if transaction_type == "deposit":
            command = DepositCommand(amount)
        elif transaction_type == "withdraw":
            command = WithdrawCommand(amount)
        else:
            raise ValueError(f"Unknown transaction type: {transaction_type}")

        # Get current balance through event replay
        current_balance = self._replay_events()

        # Execute command to get new balance
        new_balance, event_type = command.execute(current_balance)

        # Create and store immutable event
        event = BankEvent(
            transaction_id=transaction_id,
            type=event_type,
            amount=amount,
            resulting_balance=new_balance
        )

        self.event_store.append_event(event)

    def _replay_events(self) -> int:
        """Replay all events to calculate current balance."""
        balance = 0
        for event in self.event_store.get_all_events():
            balance = event.resulting_balance
        return balance

    def get_final_balance(self) -> int:
        """Get current balance by replaying all events."""
        return self._replay_events()

    def get_transaction_count(self) -> int:
        """Get total number of processed transactions."""
        return self.event_store.get_event_count()

    def get_event_log(self) -> List[Dict[str, Any]]:
        """Get complete event log as list of dictionaries."""
        events = self.event_store.get_all_events()
        return [
            {
                "transaction_id": event.transaction_id,
                "type": event.type,
                "amount": event.amount,
                "resulting_balance": event.resulting_balance
            }
            for event in events
        ]

    def get_balance_history(self) -> List[int]:
        """Get balance history after each transaction."""
        events = self.event_store.get_all_events()
        return [event.resulting_balance for event in events]


def process_bank_transactions(
        transactions: List[Tuple[str, int, int]]) -> Dict[str, Any]:
    """
    Process bank transactions using event-driven architecture.

    Args:
        transactions: List of tuples (type, amount, transaction_id)

    Returns:
        Dictionary with final_balance, transaction_count, event_log,
        balance_history
    """
    # Create bank account instance
    account = BankAccount()

    # Process each transaction
    for transaction_type, amount, transaction_id in transactions:
        account.process_transaction(transaction_type, amount, transaction_id)

    # Build and return result
    return {
        "final_balance": account.get_final_balance(),
        "transaction_count": account.get_transaction_count(),
        "event_log": account.get_event_log(),
        "balance_history": account.get_balance_history()
    }


# Example usage and testing
if __name__ == "__main__":
    # Test Example 1
    transactions1 = [("deposit", 1000, 1), ("withdraw", 300, 2)]
    result1 = process_bank_transactions(transactions1)
    print(result1)
    print()

    # Test Example 2
    transactions2 = [("deposit", 500, 1), ("deposit", 200, 2), ("withdraw", 100, 3)]
    result2 = process_bank_transactions(transactions2)
    print(result2)
    print()


In [ ]:
# tests
"""Unit tests for the event-driven banking system."""

import unittest
from main import process_bank_transactions


class TestBankAccountSystem(unittest.TestCase):
    """Test suite for bank account operations and event replay."""

    def test_single_deposit(self):
        """Test processing a single deposit."""
        tx = [("deposit", 100, 1)]
        result = process_bank_transactions(tx)
        self.assertEqual(result["final_balance"], 100)
        self.assertEqual(result["transaction_count"], 1)
        self.assertEqual(result["balance_history"], [100])

    def test_single_withdrawal(self):
        """Test a deposit followed by a withdrawal."""
        tx = [("deposit", 200, 1), ("withdraw", 50, 2)]
        result = process_bank_transactions(tx)
        self.assertEqual(result["final_balance"], 150)
        self.assertEqual(result["transaction_count"], 2)
        self.assertEqual(result["balance_history"], [200, 150])

    def test_multiple_deposits(self):
        """Test multiple deposit transactions."""
        tx = [("deposit", 100, 1), ("deposit", 200, 2)]
        result = process_bank_transactions(tx)
        self.assertEqual(result["final_balance"], 300)
        self.assertEqual(result["transaction_count"], 2)

    def test_multiple_withdrawals(self):
        """Test deposit followed by multiple withdrawals."""
        tx = [
            ("deposit", 500, 1),
            ("withdraw", 100, 2),
            ("withdraw", 50, 3)
        ]
        result = process_bank_transactions(tx)
        self.assertEqual(result["final_balance"], 350)
        self.assertEqual(result["balance_history"], [500, 400, 350])

    def test_zero_balance(self):
        """Test when no transactions are provided."""
        tx = []
        result = process_bank_transactions(tx)
        self.assertEqual(result["final_balance"], 0)
        self.assertEqual(result["transaction_count"], 0)
        self.assertEqual(result["balance_history"], [])

    def test_negative_balance(self):
        """Test starting with a withdrawal to get negative balance."""
        tx = [("withdraw", 100, 1)]
        result = process_bank_transactions(tx)
        self.assertEqual(result["final_balance"], -100)

    def test_alternating_transactions(self):
        """Test alternating deposits and withdrawals."""
        tx = [
            ("deposit", 300, 1),
            ("withdraw", 100, 2),
            ("deposit", 200, 3),
            ("withdraw", 50, 4)
        ]
        result = process_bank_transactions(tx)
        self.assertEqual(result["final_balance"], 350)
        self.assertEqual(result["balance_history"], [300, 200, 400, 350])

    def test_event_log_structure(self):
        """Test structure of event log entries."""
        tx = [("deposit", 150, 1)]
        result = process_bank_transactions(tx)
        self.assertIsInstance(result["event_log"], list)
        self.assertEqual(result["event_log"][0]["transaction_id"], 1)
        self.assertEqual(result["event_log"][0]["type"], "deposit")
        self.assertEqual(result["event_log"][0]["amount"], 150)
        self.assertEqual(result["event_log"][0]["resulting_balance"], 150)

    def test_transaction_ids_preserved(self):
        """Test that transaction IDs are preserved."""
        tx = [
            ("deposit", 100, 11),
            ("withdraw", 50, 22),
            ("deposit", 30, 33)
        ]
        result = process_bank_transactions(tx)
        self.assertEqual(
            [e["transaction_id"] for e in result["event_log"]],
            [11, 22, 33]
        )

    def test_negative_deposit(self):
        """Test depositing a negative amount."""
        tx = [("deposit", -100, 1)]
        result = process_bank_transactions(tx)
        self.assertEqual(result["final_balance"], -100)

    def test_negative_withdrawal(self):
        """Test withdrawing a negative amount (effectively deposits)."""
        tx = [("withdraw", -50, 1)]
        result = process_bank_transactions(tx)
        self.assertEqual(result["final_balance"], 50)

    def test_large_amounts(self):
        """Test large deposits and withdrawals."""
        tx = [("deposit", 1_000_000, 1), ("withdraw", 999_999, 2)]
        result = process_bank_transactions(tx)
        self.assertEqual(result["final_balance"], 1)

    def test_duplicate_transaction_ids(self):
        """Test system behavior with duplicate transaction IDs."""
        tx = [
            ("deposit", 100, 1),
            ("withdraw", 50, 1)
        ]
        result = process_bank_transactions(tx)
        self.assertEqual(result["transaction_count"], 2)

    def test_unknown_transaction_type(self):
        """Test exception is raised on unknown transaction type."""
        tx = [("invalid", 100, 1)]
        with self.assertRaises(ValueError):
            process_bank_transactions(tx)

    def test_no_mutation_of_event_log(self):
        """Modifying returned event log doesn't affect internal state."""
        tx = [("deposit", 100, 1)]
        result = process_bank_transactions(tx)
        log1 = result["event_log"]
        log1[0]["amount"] = 999
        result2 = process_bank_transactions(tx)
        self.assertEqual(result2["event_log"][0]["amount"], 100)

    def test_event_count_matches_transaction_count(self):
        """Test event log and transaction count match."""
        tx = [
            ("deposit", 10, 1),
            ("deposit", 20, 2),
            ("withdraw", 5, 3)
        ]
        result = process_bank_transactions(tx)
        self.assertEqual(len(result["event_log"]), 3)
        self.assertEqual(result["transaction_count"], 3)

    def test_balance_history_correctness(self):
        """Test correctness of intermediate balances."""
        tx = [
            ("deposit", 10, 1),
            ("deposit", 20, 2),
            ("withdraw", 5, 3)
        ]
        result = process_bank_transactions(tx)
        self.assertEqual(result["balance_history"], [10, 30, 25])

    def test_order_preservation(self):
        """Test order of transactions in event log."""
        tx = [
            ("deposit", 100, 1),
            ("withdraw", 10, 2),
            ("deposit", 40, 3)
        ]
        result = process_bank_transactions(tx)
        self.assertEqual(
            [e["type"] for e in result["event_log"]],
            ["deposit", "withdraw", "deposit"]
        )

    def test_empty_transaction_list(self):
        """Test result with no input transactions."""
        result = process_bank_transactions([])
        self.assertEqual(result["final_balance"], 0)
        self.assertEqual(result["transaction_count"], 0)
        self.assertEqual(result["event_log"], [])
        self.assertEqual(result["balance_history"], [])

    def test_mixed_zero_and_nonzero(self):
        """Test deposits and withdrawals of zero and nonzero amounts."""
        tx = [
            ("deposit", 0, 1),
            ("deposit", 100, 2),
            ("withdraw", 0, 3),
            ("withdraw", 100, 4)
        ]
        result = process_bank_transactions(tx)
        self.assertEqual(result["final_balance"], 0)
        self.assertEqual(result["transaction_count"], 4)
        self.assertEqual(result["balance_history"], [0, 100, 100, 0])

    def test_repeated_transactions(self):
        """Test processing of repeated deposit transactions."""
        tx = [("deposit", 50, i) for i in range(10)]
        result = process_bank_transactions(tx)
        self.assertEqual(result["final_balance"], 500)
        self.assertEqual(result["transaction_count"], 10)


# Model Breaking Proof

#### Model Breaking Task URL: https://labeling-ms.turing.com/conversations/68002/view

#### Model code:

```python
def process_bank_transactions(transactions):
    # <Issue>: Direct balance storage violates implicit requirements
    #  and event-driven architecture
    # Initial setup
    balance = 0
    event_log = []
    balance_history = []
    
    # Process each transaction in order
    for transaction in transactions:
        trans_type, amount, trans_id = transaction
        
        # <Issue>: Direct balance manipulation instead of event replay
        # Apply the transaction
        if trans_type == "deposit":
            balance += amount
        elif trans_type == "withdraw":
            balance -= amount
        # <Issue>: Missing else clause for invalid transaction types
        
        # <Issue>: Events are not immutable - stored as mutable dictionaries
        # Create an event entry
        event = {
            "transaction_id": trans_id,
            "type": trans_type,
            "amount": amount,
            "resulting_balance": balance
        }
        
        # Record the event and resulting balance
        event_log.append(event)
        balance_history.append(balance)
    # <Issue>: Missing Architecture Components: No EventStore, Command pattern,
    #  or proper event management

    # Prepare the final output
    return {
        # <Issue>: Final balance from direct calculation, not event replay
        "final_balance": balance,
        "transaction_count": len(transactions),
        "event_log": event_log,
        "balance_history": balance_history
    }

```